# Imports

In [17]:
import zipfile
import json
from scipy import sparse
import numpy as np
import time


# Dataset path

In [18]:
datadirectory="/home/leboudy/Downloads/lastfm_train.zip"
dataset = zipfile.ZipFile(datadirectory)

# global variables initialization


In [19]:
songId=0
t=0
g=50
beta=0.2
categories =["Technical Death Metal"]
S =0#[]
id2songJsonMap = []
id2songIdMap = {}
fhat=[]
n=5 #top songs to report
#limitingCounter = 1000


# Loading data


In [ ]:
start_time = time.time()
for i in dataset.namelist():
    if  i.endswith(".json"): #songId<limitingCounter and 
        f = json.loads(dataset.read(i).decode("utf-8"))
        if  f["track_id"] not in id2songIdMap.keys():
            id2songIdMap[f["track_id"]] = songId
            tempcategories = set(categories)#will keep removing tags from it
            newtags=[]
            for tag in f["tags"]:#looping over tags
                if int(tag[1]) >g:#filters out those <g
                    newtags.append(tag)#keeping >g
                    if tag[0] in tempcategories:#if it is in the tempcategories
                        tempcategories.remove(tag[0])#removing it
            f["tags"] = newtags
            id2songJsonMap.append(f)
            #songTags= [t[0] for t in f["tags"]]
            #if len(list(filter(lambda x: x in categories,songTags )) ) == len(categories):
            if len(tempcategories)==0:#meaning we encountered all of the tags
                S+=1#.append(songId)
                fhat.append(1)
            else:
                fhat.append(0)
            songId+=1 
            
print("--- %s seconds ---" % (time.time() - start_time))

# initializing the teleportation probablilities


In [8]:
#_fhat=np.array()
#_fhat.shape=(songId,1)
#fhat=(1/len(S))*np.array(fhat)
fhat=(1/S)*np.array(fhat)

# Building the glorious Adjacency matrix


In [9]:
start_time = time.time()
col=[]
row=[]
data=[]
for i in range( len(id2songJsonMap)):
    for s in id2songJsonMap[i]["similars"]:
        if s[0] in id2songIdMap.keys() and s[1]>=t:
            col.append(i)
            row.append(id2songIdMap[s[0]])
            data.append(1)
    
    
    

        
print("--- %s seconds ---" % (time.time() - start_time))        


In [10]:
A = sparse.csr_matrix((data, (row, col)),shape=(songId, songId))

# Randomly initializing r


In [11]:
r =np.random.rand(A.shape[0])
r=r/np.sum(r)

In [12]:
print(len(id2songJsonMap))
print (A.shape)
print(songId)
print (S)
print (songId)

9330
(9330, 9330)
9330
1
9330


In [13]:
#r = beta * A.multiply(r)+(1-beta)*fhat

# Power Iterations


In [14]:
for i in range(100):
    r=A.multiply(beta).dot(r)+fhat*(1-beta)

# Reporting the top n songs


In [15]:
topIndecs = r.argsort()[-n:][::-1]


In [16]:
for item in topIndecs:
    print (id2songJsonMap[item]["track_id"])

TRACLRS12903CE9386
TRAZDQQ128F93590E2
TRBGKZD12903D13D23
TRAQTYT12903D134AC
TRAYWHE128F92E9D6A


In [70]:
print (topIndecs)

[ 197 9294 5375 6523 2346]
